# Creación del dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
path_original = "..\\data\\original\\"
path_proc =  "..\\data\\processed\\"
path_ext = "..\\data\\external\\"

## Selección de variables

In [3]:
# Lista de variables seleccionadas
sel = pd.read_csv(path_ext + "feat_selection.csv")
# Convertir a lists
sel = sel.Features.values.tolist()

In [4]:
len(sel)

58

In [5]:
# Dataset de trabajo
data = pd.read_csv(path_original + "data_original.csv")
data.head()

,FPX,FMX,HHX,INTV_QRT,WTIA_SA,WTFA_SA,SEX,HISPAN_I,R_MARITL,MRACRPI2,...,DEP_2,PAIN_4,TIRED_1,TIRED_2,TIRED_3,RCS_AFD,PAIN_2A,ANX_3R,DEP_3R,COGCAUS2
0,1,1,1,1,4228.8,3915,2,12,4,1,...,2,NaN,1,NaN,NaN,4,1,1.0,NaN,NaN
1,1,1,6,1,14650.9,16978,1,12,1,1,...,2,NaN,2,1.0,2.0,4,1,2.0,1.0,NaN
2,1,1,8,1,7066.1,10385,1,12,1,1,...,2,3.0,2,2.0,3.0,4,4,NaN,NaN,NaN
3,1,1,9,1,4497.5,3958,1,12,4,11,...,2,2.0,2,1.0,3.0,4,4,2.0,3.0,7.0
4,1,1,10,1,5594.0,6483,1,12,1,1,...,2,1.0,1,NaN,NaN,4,4,3.0,NaN,NaN


In [6]:
# Selección de variables
data = data[sel]
data.head()

,SRVY_YR,HHX,FMX,FPX,REGION,SEX,RACERPI2,AGE_P,R_MARITL,DOINGLWA,...,AHEIGHT,AWEIGHTP,BMI,APOX,AHEP,LIVEV,ASICPUSE,ASISLEEP,PROXYSA,AFLHC29_
0,2018,1,1,1,3,2,1,79,4,5,...,62,129,2358,1,2,2,1,8,2,2.0
1,2018,6,1,1,3,1,1,37,1,1,...,71,235,3279,1,2,2,4,5,2,NaN
2,2018,8,1,1,2,1,1,29,1,1,...,96,996,4363,1,2,2,4,7,2,NaN
3,2018,9,1,1,3,1,4,75,4,5,...,66,138,2229,1,2,2,1,5,2,2.0
4,2018,10,1,1,1,1,1,39,1,1,...,71,170,2372,1,2,2,4,8,2,2.0


## Creación de variable ID

El ID único de cada individuo se crea mediante la concatenación de los siguientes campos:
<ul>
    <li><strike><b>SRVY_YR:</b> año de la encuesta</strike></li>
    <li><b>HHX:</b> número de vivienda</li>
    <li><b>FMX:</b> número de familia</li>
    <li><b>FPX:</b> número de inviduo dentro de la familia</li>
</ul>
(mientras sean datos de un único año no se usará el año como ID)

In [7]:
# data["id"] = data["SRVY_YR"].astype(str) + data["HHX"].astype(str) + data["FMX"].astype(str) + data["FPX"].astype(str)

In [8]:
# Nuevo Id
data["id"] = data["HHX"].astype(str) + data["FMX"].astype(str) + data["FPX"].astype(str)
# Se descartan las variables usadas
data = data.drop(["SRVY_YR","HHX","FMX","FPX"], axis=1)

In [9]:
# Chequea que sean valores únicos
print("Cantidad de registros: " + str(data.shape[0]))
print("Cantidad de id únicos: " + str(len(np.unique(data["id"]))))

Cantidad de registros: 25417
Cantidad de id únicos: 25417


## Variable Target

La variable target es **AMIGR**: "¿ha padecido migraña o cefaleas severas en los últimos 3 meses?". Los registros con valores 7 (se rehusa a responder), 8 (no corresponde) y 9 (no sabe responder) en esta variable son removidos. Los valores 2 (no) se cambian a 0.

In [10]:
data = data[(data["AMIGR"] == 1) | (data["AMIGR"] == 2)]

In [11]:
data.loc[(data["AMIGR"] == 2),"AMIGR"] = 0

In [12]:
print("Cantidad de registros: " + str(data.shape[0]))
print("Cantidad de columnas: " + str(data.shape[1]))

Cantidad de registros: 25403
Cantidad de columnas: 55


In [13]:
# Chequea categorías
data.AMIGR.unique()

array([0, 1], dtype=int64)

In [14]:
# Rename target
data = data.rename(columns={"AMIGR":"target"})

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25403 entries, 0 to 25416
Data columns (total 55 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   REGION    25403 non-null  int64  
 1   SEX       25403 non-null  int64  
 2   RACERPI2  25403 non-null  int64  
 3   AGE_P     25403 non-null  int64  
 4   R_MARITL  25403 non-null  int64  
 5   DOINGLWA  25403 non-null  int64  
 6   WRKLYR4   25403 non-null  int64  
 7   HYPYR1    8045 non-null   float64
 8   HYPMED2   8225 non-null   float64
 9   CHLYR     7921 non-null   float64
 10  CHLMDNW2  5969 non-null   float64
 11  ANGEV     25403 non-null  int64  
 12  MIEV      25403 non-null  int64  
 13  HRTEV     25403 non-null  int64  
 14  STREV     25403 non-null  int64  
 15  EPHEV     25403 non-null  int64  
 16  COPDEV    25403 non-null  int64  
 17  ASPMEDAD  5954 non-null   float64
 18  ASPONOWN  11692 non-null  float64
 19  AASSTILL  3444 non-null   float64
 20  ULCYR     1795 non-null   fl

## Selección de la población de estudio

El dataset a utilizar contiene la información relevada de adultos mayores de 18 años. Se considerarán sólo los pacientes que:
<ul>
    <li>No padezcan enfermedades mentales graves que provoquen la pérdida del contacto con la realidad, tales como esquizofrenia</li>
    <li>No padezcan sindromes que deterioren las capacidades psíquicas, tales como la demencia senil</li>
    <li>No padezcan cáncer al momento de la encuesta</li>
    <li>No se encuentren embarazadas</li>
    <li>No padezcan addicción a sustancias de abuso como alcohol o drogas</li>
</ul>
Se utilzan las siguientes variables para filtrar la población de estudio:
<ul>
    <li>Conservar CANEV == 2. Pacientes que nunca hayan sido diagnosticados con cáncer.</li>
    <li>Descartar PREGNOW == 1. Pacientes actualmente embarazadas.</li>
    <li>Descartar PROXYSA == 1. Pacientes con discapacidad física o mental que les prohíbe responder la encuesta
        por sí mismos.</li>
    <li>Descartar AFLHC29_ == 1, 2 y 7. Pacientes con algún tipo de limitación física como consecuencia del uso de alcohol o drogas.
</ul>

In [16]:
# Filtrado
data = data[(data["CANEV"] == 2) & (data["PREGNOW"] != 1) & (data["PROXYSA"] != 1) & (data["AFLHC29_"].isin([1,2,7]) == False)]

In [17]:
# Descarta variables usadas
data = data.drop(["CANEV","PREGNOW","PROXYSA","AFLHC29_"], axis=1)

In [18]:
print("Cantidad de registros: " + str(data.shape[0]))
print("Cantidad de columnas: " + str(data.shape[1]))

Cantidad de registros: 13479
Cantidad de columnas: 51


## Recodificación de variables

### Estado civil

In [19]:
# 0: menor de 14 años. No debería haber
data[data["R_MARITL"] == 0]

,REGION,SEX,RACERPI2,AGE_P,R_MARITL,DOINGLWA,WRKLYR4,HYPYR1,HYPMED2,CHLYR,...,ALCAMT,AHEIGHT,AWEIGHTP,BMI,APOX,AHEP,LIVEV,ASICPUSE,ASISLEEP,id


In [20]:
# 1-2-3 => 1 (casado)
data.loc[((data["R_MARITL"] == 1) | (data["R_MARITL"] == 2) | (data["R_MARITL"] == 3)), "R_MARITL"] = np.int(1)
# 4 => 2 (viudo)
data.loc[data["R_MARITL"] == 4, "R_MARITL"] = np.int(2)
# 5-6 => 3 (separado)
data.loc[((data["R_MARITL"] == 5) | (data["R_MARITL"] == 6)), "R_MARITL"] = np.int(3)
# 7 => 4 (soltero)
data.loc[data["R_MARITL"] == 7, "R_MARITL"] = np.int(4)
# 8 => 5 (en pareja)
data.loc[data["R_MARITL"] == 8, "R_MARITL"] = np.int(5)
# 9 => NaN (desconocido)
data.loc[data["R_MARITL"] == 9, "R_MARITL"] = np.nan

In [21]:
# Chequea categorías
data.R_MARITL.unique()

array([ 1.,  3.,  4.,  5.,  2., nan])

### Situación laboral

#### Actual

¿Tiene trabajo actualmente?

In [22]:
# 1,2,4 => 1 (si)
data.loc[((data["DOINGLWA"] == 1) | (data["DOINGLWA"] == 2) | (data["DOINGLWA"] == 4)), "DOINGLWA"] = np.int(1)
# 3,5 => 0 (no)
data.loc[((data["DOINGLWA"] == 3) | (data["DOINGLWA"] == 5)), "DOINGLWA"] = np.int(0)
# 7,8,9 => NaN (desconocido)
data.loc[((data["DOINGLWA"] == 7) | (data["DOINGLWA"] == 8) | (data["DOINGLWA"] == 9)), "DOINGLWA"] = np.nan

In [23]:
# Chequea categorías
data.DOINGLWA.unique()

array([ 1.,  0., nan])

#### Últimos 12 meses

¿Tuvo trabajo en los últimos 12 meses?

In [24]:
# 0,1 => 1 (si)
data.loc[((data["WRKLYR4"] == 0) | (data["WRKLYR4"] == 1)), "WRKLYR4"] = np.int(1)
# 2,3 => 0 (no)
data.loc[((data["WRKLYR4"] == 2) | (data["WRKLYR4"] == 3)), "WRKLYR4"] = np.int(0)
# 7,8,9 => NaN (desconocido)
data.loc[((data["WRKLYR4"] == 7) | (data["WRKLYR4"] == 8) | (data["WRKLYR4"] == 9)), "WRKLYR4"] = np.nan

In [25]:
# Chequea categorías
data.WRKLYR4.unique()

array([ 1.,  0., nan])

### Medicamentos

#### Aspirina

Se unificarán las siguientes variables:
<ul>
    <li><b>ASPMEDAD</b>: toma aspirina por recomendación médica.</li>
    <li><b>ASPONOWN</b>: toma aspirina sin recomendación médica.</li>
</ul>
Considerándose la pregunta: "¿Toma aspirina actualmente?". Los NaN correspondena individuos que no cumplen con la población en estudio (adultos mayores de 40 que alguna vez se les ha recomendado que tomen aspirina).

In [26]:
# Los NaN son convertidos a 0, dado que no cumplen con la condición de la población en estudio.
data.loc[(data["ASPMEDAD"].isna() == True), "ASPMEDAD"] = np.int(0)
data.loc[(data["ASPONOWN"].isna() == True), "ASPONOWN"] = np.int(0)

In [27]:
data[["ASPMEDAD","ASPONOWN"]]

,ASPMEDAD,ASPONOWN
2,0.0,0.0
5,0.0,2.0
7,0.0,2.0
8,0.0,0.0
13,2.0,0.0
...,...,...
25405,0.0,0.0
25407,0.0,0.0
25410,0.0,0.0
25411,0.0,0.0


In [28]:
conditions = [
    (data['ASPMEDAD'] == 1) | (data['ASPONOWN'] == 1),   # una de las respuestas es "si"
    (data['ASPMEDAD'] == 2) | (data['ASPONOWN'] == 2),   # la respuesta es "no"
    (data['ASPMEDAD'] == 0) & (data['ASPONOWN'] == 0)]   # la respuesta es "no" por no ser parte de la población

choices = [1, 0, 0]

data['ASP'] = np.select(conditions, choices, default=np.nan)

In [29]:
data[["ASPMEDAD","ASPONOWN", "ASP"]].head()

,ASPMEDAD,ASPONOWN,ASP
2,0.0,0.0,0.0
5,0.0,2.0,0.0
7,0.0,2.0,0.0
8,0.0,0.0,0.0
13,2.0,0.0,0.0


In [30]:
# Chequea categorías
data.ASP.unique()

array([ 0.,  1., nan])

In [31]:
# Descarta variables usadas
data = data.drop(["ASPMEDAD","ASPONOWN"], axis=1)

### Enfermedades crónicas

#### Diabetes

Se reformula la pregunta: "¿alguna vez le han diagnosticado diabetes o pre-diabetes?"

In [32]:
# 1 => 1 (si)
data.loc[data["DIBEV1"] == 1, "DIBEV1"] = np.int(1)
# 2 => 0 (no)
data.loc[data["DIBEV1"] == 2, "DIBEV1"] = np.int(0)
# 3 => 1 (prediabetes)
data.loc[data["DIBEV1"] == 3, "DIBEV1"] = np.int(1)
# 7,8,9 => NaN (desconocido)
data.loc[((data["DIBEV1"] == 7) | (data["DIBEV1"] == 8) | (data["DIBEV1"] == 9)), "DIBEV1"] = np.nan

In [33]:
# Chequea categorías
data.DIBEV1.unique()

array([ 0.,  1., nan])

#### Limitaciones físicas

¿Posee algún tipo de limitación física/funcional?

In [34]:
# 1 => 1 (si)
data.loc[data["FLA1AR"] == 0, "FLA1AR"] = np.int(1)
# 2 => 0 (no)
data.loc[data["FLA1AR"] == 2, "FLA1AR"] = np.int(0)
# 3 => nan (no lo sabe)
data.loc[data["FLA1AR"] == 3, "FLA1AR"] = np.nan

In [35]:
# Chequea categorías
data.FLA1AR.unique()

array([ 0.,  1., nan])

### Hábitos

#### Uso de PC

¿Qué tan frecuente utiliza la computadora?

In [36]:
# 1 => 0 (nunca o casi nunca)
data.loc[data["ASICPUSE"] == 1, "ASICPUSE"] = np.int(0)
# 2 => 1 (a veces)
data.loc[data["ASICPUSE"] == 2, "ASICPUSE"] = np.int(1)
# 3 => 2 (casi todos los días)
data.loc[data["ASICPUSE"] == 3, "ASICPUSE"] = np.int(2)
# 4 => 3 (todos los días)
data.loc[data["ASICPUSE"] == 4, "ASICPUSE"] = np.int(3)
# 7,8,9 => NaN (desconocido)
data.loc[((data["ASICPUSE"] == 7) | (data["ASICPUSE"] == 8) | (data["ASICPUSE"] == 9)), "ASICPUSE"] = np.nan

In [37]:
# Chequea categorías
data.ASICPUSE.unique()

array([ 3.,  1.,  0.,  2., nan])

#### Horas de sueño

En promedio ¿cuántas horas duerme en un periodo de 24 horas?

In [38]:
# 97,98,99 => nan (desconocido)
data.loc[((data["ASISLEEP"] == 97) | (data["ASISLEEP"] == 98) | (data["ASISLEEP"] == 99)), "ASISLEEP"] = np.nan

In [39]:
# Chequea categorías
data.ASISLEEP.unique()

array([ 7.,  6.,  8.,  4.,  5.,  9., 10., nan,  3., 14., 12., 11., 18.,
        2., 13., 16., 15.,  1.])

#### Fumar

¿Cuántos cigarros (de cualquier tipo) fuma al día?. Esta pregunta es sólo para fumadores, los NaN son no fumadores.

In [40]:
# Cantidad de cigarrillos que fuma por día (todo los tipos)
# 97,98,99 => nan (desconocido)
data.loc[((data["CIGSDAY"] == 97) | (data["CIGSDAY"] == 98) | (data["CIGSDAY"] == 99)), "AHEIGHT"] = np.nan

In [41]:
# Los NaN son convertidos a 0, dado que no cumplen con la condición de la población en estudio.
data.loc[(data["CIGSDAY"].isna() == True), "CIGSDAY"] = np.int(0)

In [42]:
# Chequea categorías
data.CIGSDAY.unique()

array([ 0.,  8., 20.,  2., 25.,  1.,  3., 40., 60., 99., 15., 10.,  5.,
        4., 12.,  7., 18.,  6., 30., 19., 17., 13., 22.,  9., 24., 14.,
       11., 97., 16., 35., 21.])

#### Ejercicio físico

Se unifican las preguntas de "¿Cuántas veces por semana realiza ejercicico?" y "¿Cuántos minutos de ejericicio realiza?" a la pregunta: "¿Cuntos minutos de ejercico realiza por semana?". Separado según la intensidad del ejercicio:
   - **VIG**: ejercicio vigoroso.
   - **MOD**: ejercicio liviano o moderado.

**Vigoroso**

In [43]:
# Frecuencia semanal
# 00,95,96 => 0 (nunca)
data.loc[((data["VIGFREQW"] == 0) | (data["VIGFREQW"] == 95) | (data["VIGFREQW"] == 96)), "VIGFREQW"] = np.int(0)
# 97,98,99 => nan (desconocido)
data.loc[((data["VIGFREQW"] == 97) | (data["VIGFREQW"] == 98) | (data["VIGFREQW"] == 99)), "VIGFREQW"] = np.nan

# Minutos
# 997,998,999 => nan (desconocido)
data.loc[((data["VIGMIN"] == 997) | (data["VIGMIN"] == 998) | (data["VIGMIN"] == 999)), "VIGMIN"] = np.nan

In [44]:
# Condiciones para calcular los minutos de ejercicio
conditions = [
    (data['VIGFREQW'] == 0),             # no realiza ejercicio
    (data['VIGFREQW'].isna() == True),   # Se desconoce si realiza ejercicio
    (data['VIGFREQW'] > 0)]              # Realiza ejercicio más de 1 vez por semana

choices = [0, np.nan, data['VIGFREQW'] * data['VIGMIN']]

data['VIG'] = np.select(conditions, choices, default=0)

In [45]:
data[["VIGFREQW","VIGMIN","VIG"]]

,VIGFREQW,VIGMIN,VIG
2,4.0,30.0,120.0
5,0.0,NaN,0.0
7,14.0,15.0,210.0
8,3.0,60.0,180.0
13,5.0,90.0,450.0
...,...,...,...
25405,1.0,20.0,20.0
25407,2.0,30.0,60.0
25410,0.0,NaN,0.0
25411,4.0,60.0,240.0


In [46]:
data.loc[data["VIG"] > 5000, ["VIGFREQW","VIGMIN","VIG","AGE_P"]]

,VIGFREQW,VIGMIN,VIG,AGE_P
579,7.0,720.0,5040.0,46
2079,21.0,240.0,5040.0,20
3197,28.0,180.0,5040.0,20
12531,14.0,480.0,6720.0,23
13499,21.0,480.0,10080.0,57
20107,7.0,720.0,5040.0,30
20369,14.0,480.0,6720.0,66
21987,15.0,600.0,9000.0,53
24382,28.0,180.0,5040.0,57


In [47]:
# Se descartan las variables viejas
data = data.drop(["VIGFREQW","VIGMIN"], axis=1)

In [48]:
# Chequea categorías
data.VIG.unique()

array([1.200e+02, 0.000e+00, 2.100e+02, 1.800e+02, 4.500e+02, 5.600e+02,
       7.200e+02, 1.500e+02, 9.000e+01, 2.400e+02, 8.000e+01, 3.000e+01,
       6.000e+01, 1.350e+02, 1.000e+02, 1.050e+02, 1.600e+02, 3.000e+02,
       4.200e+02, 1.500e+01, 1.400e+02, 4.500e+01, 6.300e+02, 8.400e+02,
       6.000e+02, 3.150e+02, 2.250e+02, 3.600e+02, 5.000e+01, 2.700e+02,
       2.000e+01, 2.520e+03, 2.520e+02,       nan, 1.820e+02, 7.000e+01,
       4.200e+01, 4.800e+02, 3.360e+03, 2.800e+02, 1.260e+03, 1.680e+03,
       5.040e+03, 1.750e+02, 2.000e+02, 1.890e+03, 7.500e+01, 2.500e+02,
       1.000e+01, 2.500e+01, 5.400e+02, 9.000e+02, 4.000e+01, 1.250e+02,
       1.440e+03, 4.900e+02, 3.300e+02, 3.750e+02, 1.120e+03, 2.750e+02,
       9.450e+02, 1.200e+03, 2.450e+02, 9.800e+02, 6.800e+01, 2.100e+03,
       1.080e+03, 5.250e+02, 3.500e+01, 1.540e+02, 3.500e+02, 1.500e+03,
       1.335e+03, 3.780e+03, 3.200e+02, 9.600e+02, 1.400e+03, 1.060e+02,
       1.320e+03, 2.200e+02, 1.650e+02, 6.720e+03, 

**Moderado**

In [49]:
# Frecuencia semanal
# 00,95,96 => 0 (nunca)
data.loc[((data["MODFREQW"] == 0) | (data["MODFREQW"] == 95) | (data["MODFREQW"] == 96)), "MODFREQW"] = np.int(0)
# 97,98,99 => nan (desconocido)
data.loc[((data["MODFREQW"] == 97) | (data["MODFREQW"] == 98) | (data["MODFREQW"] == 99)), "MODFREQW"] = np.nan

# Minutos
# 997,998,999 => nan (desconocido)
data.loc[((data["MODMIN"] == 997) | (data["MODMIN"] == 998) | (data["MODMIN"] == 999)), "MODMIN"] = np.nan

In [50]:
# Condiciones para calcular los minutos de ejercicio
conditions = [
    (data['MODFREQW'] == 0),             # no realiza ejercicio
    (data['MODFREQW'].isna() == True),   # Se desconoce si realiza ejercicio
    (data['MODFREQW'] > 0)]              # Realiza ejercicio más de 1 vez por semana

choices = [0, np.nan, data['MODFREQW'] * data['MODMIN']]

data['MOD'] = np.select(conditions, choices, default=0)

In [51]:
data[["MODFREQW","MODMIN","MOD"]]

,MODFREQW,MODMIN,MOD
2,7.0,30.0,210.0
5,0.0,NaN,0.0
7,14.0,15.0,210.0
8,4.0,30.0,120.0
13,0.0,NaN,0.0
...,...,...,...
25405,3.0,10.0,30.0
25407,1.0,30.0,30.0
25410,1.0,30.0,30.0
25411,2.0,30.0,60.0


In [52]:
data.loc[data["MOD"] > 5000, ["MODFREQW","MODMIN","MOD","AGE_P"]]

,MODFREQW,MODMIN,MOD,AGE_P
825,14.0,360.0,5040.0,60
1883,21.0,240.0,5040.0,50
6829,28.0,180.0,5040.0,73
7617,28.0,180.0,5040.0,60
10238,7.0,720.0,5040.0,24
12127,21.0,300.0,6300.0,65
15401,28.0,360.0,10080.0,31
15879,28.0,240.0,6720.0,46
19597,7.0,720.0,5040.0,61
20152,7.0,720.0,5040.0,48


In [53]:
# Se descartan las variables viejas
data = data.drop(["MODFREQW","MODMIN"], axis=1)

In [54]:
# Chequea categorías
data.MOD.unique()

array([2.100e+02, 0.000e+00, 1.200e+02, 5.600e+02, 1.500e+02, 2.800e+02,
       9.000e+01, 8.400e+02, 3.000e+02, 1.600e+02, 2.250e+02, 1.750e+02,
       6.000e+01, 6.000e+02, 1.050e+02, 1.800e+02, 1.000e+02, 4.000e+01,
       3.000e+01, 7.000e+01, 5.000e+01,       nan, 4.200e+02, 2.400e+02,
       4.500e+01, 2.700e+02, 3.600e+02, 1.400e+02, 2.000e+01, 3.150e+02,
       1.350e+02, 7.500e+01, 1.260e+03, 3.360e+03, 2.940e+03, 1.000e+01,
       1.680e+03, 2.000e+02, 1.200e+01, 1.500e+01, 5.040e+03, 9.600e+02,
       6.300e+02, 8.000e+01, 2.520e+03, 1.250e+02, 3.500e+02, 5.250e+02,
       2.450e+02, 4.800e+02, 9.000e+02, 2.100e+03, 4.900e+02, 7.200e+02,
       3.400e+02, 9.450e+02, 9.800e+01, 2.400e+03, 4.500e+02, 4.600e+01,
       1.920e+03, 3.500e+01, 5.400e+02, 1.200e+03, 8.400e+01, 1.080e+03,
       2.500e+01, 4.000e+02, 1.500e+03, 2.160e+03, 2.500e+02, 4.320e+03,
       3.900e+01, 1.800e+03, 8.500e+01, 1.120e+03, 3.750e+02, 4.200e+03,
       6.300e+03, 1.610e+02, 5.400e+01, 1.440e+03, 

#### Consumo de alcohol

Se unifican las preguntas "¿En promedio cuántas veces por semana bebió alcohol en el último año?" y "¿En promedio cuántos vasos bebe cada vez que toma?" a: ¿En promedio cuántos vasos de alcohol bebe por semana?"

**Bebidas por semana**

In [55]:
# 00,95 => 0 (nunca)
data.loc[((data["ALC12MWK"] == 0) | (data["ALC12MWK"] == 95)), "ALC12MWK"] = np.int(0)
# 97,98,99 => nan (desconocido)
data.loc[((data["ALC12MWK"] == 97) | (data["ALC12MWK"] == 98) | (data["ALC12MWK"] == 99)), "ALC12MWK"] = np.nan
# NaN => 0 (No bebe)
data.loc[(data['ALC12MWK'].isna() == True), "ALC12MWK"] = np.int(0)

In [56]:
# Chequea categorías
data.ALC12MWK.unique()

array([0., 1., 3., 2., 6., 5., 4., 7.])

**Cantidad de vasos**

In [57]:
# 97,98,99 => nan (desconocido)
data.loc[((data["ALCAMT"] == 97) | (data["ALCAMT"] == 98) | (data["ALCAMT"] == 99)), "ALCAMT"] = np.nan
# NaN => 0 (No bebió en el último año)
data.loc[(data['ALCAMT'].isna() == True), "ALCAMT"] = np.int(0)

In [58]:
# Chequea categorías
data.ALCAMT.unique()

array([ 1.,  0.,  2.,  3.,  4.,  6.,  8.,  5., 12., 10.,  7., 30., 15.,
        9., 18., 20., 13., 52., 22., 35., 24., 14., 90., 95.])

**Nueva variable**

In [59]:
# Condiciones para calcular los minutos de ejercicio
conditions = [
    (data['ALC12MWK'] == 0),             # no realiza ejercicio
    (data['ALC12MWK'] > 0)]              # Realiza ejercicio más de 1 vez por semana

choices = [0, data['ALC12MWK'] * data['ALCAMT']]

data['ALC'] = np.select(conditions, choices, default=0)

In [60]:
data[["ALC12MWK","ALCAMT", "ALC"]]

,ALC12MWK,ALCAMT,ALC
2,0.0,1.0,0.0
5,0.0,0.0,0.0
7,0.0,0.0,0.0
8,1.0,2.0,2.0
13,0.0,0.0,0.0
...,...,...,...
25405,0.0,3.0,0.0
25407,3.0,2.0,6.0
25410,0.0,0.0,0.0
25411,0.0,4.0,0.0


In [61]:
# Chequea categorías
data.ALC.unique()

array([  0.,   2.,   6.,   4.,   1.,  12.,   8.,   5.,   3.,  16.,  14.,
        42.,   7.,  15.,  30.,   9.,  18.,  24.,  10.,  21.,  35.,  70.,
        28.,  32.,  36.,  49.,  25.,  20.,  84.,  13.,  52.,  56., 105.,
       132.,  48., 126., 245.,  40.,  72., 140., 630.,  45., 210., 285.])

In [62]:
# Se descartan las variables viejas
data = data.drop(["ALC12MWK","ALCAMT"], axis=1)

### Estado físico del paciente

#### Altura

In [63]:
# 96,97,98,99 => nan (desconocido)
data.loc[((data["AHEIGHT"] == 96) | (data["AHEIGHT"] == 97) | (data["AHEIGHT"] == 98) | (data["AHEIGHT"] == 99)), "AHEIGHT"] = np.nan

In [64]:
# Se convierte de pulgadas a cm
data["AHEIGHT"] = data["AHEIGHT"] * 2.54

In [65]:
print("Mínimo valor: " + str(data["AHEIGHT"].min()))
print("Máximo valor: " + str(data["AHEIGHT"].max()))

Mínimo valor: 149.86
Máximo valor: 193.04


#### Peso

In [66]:
# 996,997,998,999 => nan (desconocido)
data.loc[((data["AWEIGHTP"] == 996) | (data["AWEIGHTP"] == 997) | (data["AWEIGHTP"] == 998) | (data["AWEIGHTP"] == 999)), "AWEIGHTP"] = np.nan

In [67]:
# Se convierte de pounds a Kg
data["AWEIGHTP"] = round(data["AWEIGHTP"] * 0.453592, 1)

In [68]:
print("Mínimo valor: " + str(data["AWEIGHTP"].min()))
print("Máximo valor: " + str(data["AWEIGHTP"].max()))

Mínimo valor: 45.4
Máximo valor: 135.2


#### Índice de masa corporal

Esta variable si bien es contínua, está indicada en el dataset original por códigos. Aquí se reemplazará por el cálculo de BMI = peso (Kg) / altura (m) al cuadrado

In [69]:
# 9999 => nan (desconocido)
data.loc[data["BMI"] == 9999, "BMI"] = np.nan

In [70]:
data["BMI"] = round(data["AWEIGHTP"] / np.power(data["AHEIGHT"]/100,2),2)

In [71]:
print("Mínimo valor: " + str(data["BMI"].min()))
print("Máximo valor: " + str(data["BMI"].max()))

Mínimo valor: 15.68
Máximo valor: 49.92


### Variables con misma recodificación

In [73]:
feat_list = ["HYPYR1",
"HYPMED2",
"CHLYR",
"CHLMDNW2",
"ANGEV",
"MIEV",
"HRTEV",
"STREV",
"EPHEV",
"COPDEV",
"AASSTILL",
"ULCYR",
"DIBPILL1",
"INSLN1",
"AHAYFYR",
"SINYR",
"CBRCHYR",
"KIDWKYR",
"LIVYR",
"ARTH1",
"PAINECK",
"PAINLB",
"PAINFACE",
"APOX",
"AHEP",
"LIVEV"]

In [74]:
for i in feat_list:
    print("Variable " + str(i))
    print("   Categorías: " +  str(list(data[i].unique())))
    print("   Nulos: " + str(round(data[i].isna().sum()*100/len(data),2)))
    print()

Variable HYPYR1
   Categorías: [nan, 1.0, 2.0, 9.0]
   Nulos: 82.08

Variable HYPMED2
   Categorías: [nan, 1.0, 2.0]
   Nulos: 81.83

Variable CHLYR
   Categorías: [nan, 1.0, 9.0, 2.0]
   Nulos: 80.7

Variable CHLMDNW2
   Categorías: [nan, 1.0, 2.0]
   Nulos: 87.45

Variable ANGEV
   Categorías: [2, 1, 9, 7]
   Nulos: 0.0

Variable MIEV
   Categorías: [2, 1, 7, 9]
   Nulos: 0.0

Variable HRTEV
   Categorías: [2, 1, 7, 9]
   Nulos: 0.0

Variable STREV
   Categorías: [2, 1, 7]
   Nulos: 0.0

Variable EPHEV
   Categorías: [2, 1, 7, 9]
   Nulos: 0.0

Variable COPDEV
   Categorías: [2, 1, 9, 7]
   Nulos: 0.0

Variable AASSTILL
   Categorías: [nan, 1.0, 2.0, 9.0]
   Nulos: 89.14

Variable ULCYR
   Categorías: [nan, 2.0, 1.0, 9.0]
   Nulos: 96.62

Variable DIBPILL1
   Categorías: [nan, 2.0, 1.0, 9.0]
   Nulos: 89.94

Variable INSLN1
   Categorías: [nan, 2.0, 1.0]
   Nulos: 89.94

Variable AHAYFYR
   Categorías: [2, 1, 9]
   Nulos: 0.0

Variable SINYR
   Categorías: [2, 1, 9]
   Nulos: 0.0

Va

In [75]:
for i in feat_list:
    print("Transformando variable " + str(i))
    # Los NaN son convertidos a 0, dado que no cumplen con la condición de la población en estudio
    data.loc[(data[i].isna() == True), i] = np.int(0)
    # 1 => 1 (si)
    data.loc[(data[i] == 1), i] = np.int(1)
    # 2 => 0 (no)
    data.loc[(data[i] == 2), i] = np.int(0)
    # 7,8,9 => NaN (desconocido)
    data.loc[((data[i] == 7) | (data[i] == 8) | (data[i] == 9)), i] = np.nan    
    print("Categorías: " +  str(list(data[i].unique())))
    print()

Transformando variable HYPYR1
Categorías: [0.0, 1.0, nan]

Transformando variable HYPMED2
Categorías: [0.0, 1.0]

Transformando variable CHLYR
Categorías: [0.0, 1.0, nan]

Transformando variable CHLMDNW2
Categorías: [0.0, 1.0]

Transformando variable ANGEV
Categorías: [0.0, 1.0, nan]

Transformando variable MIEV
Categorías: [0.0, 1.0, nan]

Transformando variable HRTEV
Categorías: [0.0, 1.0, nan]

Transformando variable STREV
Categorías: [0.0, 1.0, nan]

Transformando variable EPHEV
Categorías: [0.0, 1.0, nan]

Transformando variable COPDEV
Categorías: [0.0, 1.0, nan]

Transformando variable AASSTILL
Categorías: [0.0, 1.0, nan]

Transformando variable ULCYR
Categorías: [0.0, 1.0, nan]

Transformando variable DIBPILL1
Categorías: [0.0, 1.0, nan]

Transformando variable INSLN1
Categorías: [0.0, 1.0]

Transformando variable AHAYFYR
Categorías: [0.0, 1.0, nan]

Transformando variable SINYR
Categorías: [0.0, 1.0, nan]

Transformando variable CBRCHYR
Categorías: [0.0, 1.0, nan]

Transformand

### Exportar dataset

In [76]:
# Reordenar
data = data[
    [
        "id",
        "REGION",
        "SEX",
        "AHEIGHT",
        "AWEIGHTP",
        "BMI",
        "AGE_P",
        "RACERPI2",
        "R_MARITL",
        "DOINGLWA",
        "WRKLYR4",
        "HYPYR1",
        "HYPMED2",
        "CHLYR",
        "CHLMDNW2",
        "ANGEV",
        "MIEV",
        "HRTEV",
        "STREV",
        "EPHEV",
        "COPDEV",
        "ASP",
        "AASSTILL",
        "ULCYR",
        "DIBEV1",
        "DIBPILL1",
        "INSLN1",
        "AHAYFYR",
        "SINYR",
        "CBRCHYR",
        "KIDWKYR",
        "LIVYR",
        "ARTH1",
        "PAINECK",
        "PAINLB",
        "PAINFACE",
        "FLA1AR",
        "CIGSDAY",
        "VIG",
        "MOD",
        "ALC",
        "APOX",
        "AHEP",
        "LIVEV",
        "ASICPUSE",
        "ASISLEEP",
        "target",
    ]
]

In [77]:
# Reset index
data = data.reset_index(drop=True)

In [78]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13479 entries, 0 to 13478
Data columns (total 47 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        13479 non-null  object 
 1   REGION    13479 non-null  int64  
 2   SEX       13479 non-null  int64  
 3   AHEIGHT   12711 non-null  float64
 4   AWEIGHTP  12497 non-null  float64
 5   BMI       12470 non-null  float64
 6   AGE_P     13479 non-null  int64  
 7   RACERPI2  13479 non-null  int64  
 8   R_MARITL  13450 non-null  float64
 9   DOINGLWA  13471 non-null  float64
 10  WRKLYR4   13459 non-null  float64
 11  HYPYR1    13459 non-null  float64
 12  HYPMED2   13479 non-null  float64
 13  CHLYR     13427 non-null  float64
 14  CHLMDNW2  13479 non-null  float64
 15  ANGEV     13468 non-null  float64
 16  MIEV      13472 non-null  float64
 17  HRTEV     13474 non-null  float64
 18  STREV     13478 non-null  float64
 19  EPHEV     13477 non-null  float64
 20  COPDEV    13477 non-null  fl

In [79]:
data.target.value_counts()

0    11951
1     1528
Name: target, dtype: int64

In [80]:
# Exportar dataset
data.to_csv(path_proc + "data_filter.csv", index=False)